# Test Our Approach

Difference between evaluation and testing: for evaluation, we always use the ground truth annotation to extract the contour points. For testing, we can only use the first ground truth annotation to extract the contour points and then need to create the next annotation by using our predicted translations.

Pipeline for testing:
- Take first ground truth annotation of sequence as input
- Extract contour_0
- Create data object
- Forward pass data through model to get translation_0_1
- Compute contour_1 = contour_0 + translation_0_1
- Get OSVOS image
- Compute combo image

## Imports

In [1]:
import os

import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch 

from pg_networks.gcn import GCN
import src.config as cfg
from src.create_data import create_osvos_model, create_data
from src.metrics import db_eval_iou, db_eval_boundary, db_eval_t_stab
from src.vis_utils import compute_combo_img, extract_longest_contour, load_gray_img

# for auto-reloading extenrnal modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

## Load OSVOS model to extract feature vectors

In [2]:
osvos_model = create_osvos_model(cfg.PARENT_MODEL_PATH, cfg.LAYER)

Constructing OSVOS architecture..
Initializing weights..


# Load GCN model

In [3]:
gcn_model_path = 'pg_models/2019-06-28_17_14_33_best_model.pth'
gcn_model = GCN(256, 2)
gcn_model.load_state_dict(torch.load(gcn_model_path))
gcn_model.eval()
gcn_model.double()

GCN(
  (conv1): GCNConv(256, 512)
  (conv2): GCNConv(512, 512)
  (conv3): GCNConv(512, 1024)
  (conv4): GCNConv(1024, 1024)
  (lin1): Linear(in_features=1024, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=2, bias=True)
)

In [4]:
mean_Js_combo = []
mean_Js_osvos = []

mean_Fs_combo = []
mean_Fs_osvos = []

# Iterate through val sequences
for i, sequence in enumerate(cfg.VAL_SEQUENCES):
    
    #if i > 0: break
    print('#{}: {}'.format(i, sequence))
    
    # Get list of frames
    raw_images_path = os.path.join('pg_datasets/DAVIS_2016/raw/Images', sequence, '0')
    raw_annotations_path = os.path.join('pg_datasets/DAVIS_2016/raw/Annotations', sequence, '0')
    
    # Get list of frames
    frames = os.listdir(raw_images_path)
    if '.ipynb_checkpoints' in frames:
        frames.remove('.ipynb_checkpoints')
    frames.sort()
    
    Js_combo = []
    Js_osvos = []

    Fs_combo = []
    Fs_osvos = []    
    
    # Iterate through frames
    for j, frame in enumerate(frames[:-1]):
        
        #if j > 5: break
        #print('\t#{}: {}'.format(j, frame))
        
        # If first frame, extract contour from gt annotation
        if j == 0:
            annotation_0_path = os.path.join(raw_annotations_path, frame[:5] + '.png')
            annotation_0_gray = load_gray_img(annotation_0_path)
            
            contour_0 = extract_longest_contour(annotation_0_gray, 
                                                cfg.CLOSING_KERNEL_SIZE, 
                                                cv2.CHAIN_APPROX_TC89_KCOS)
            contour_0 = np.squeeze(contour_0)

        # Create data object
        image_path_0 = os.path.join(raw_images_path, frames[j])
        image_path_1 = os.path.join(raw_images_path, frames[j+1])
        
        data = create_data(contour_0, None, image_path_0, image_path_1, osvos_model, cfg.K)
        
        # Forward pass to get outputs
        with torch.no_grad():
            translation_0_1_pred = gcn_model(data)    
    
        # Compute contour_1
        contour_1_pred = np.add(contour_0, translation_0_1_pred)

        # Load OSVOS result image
        osvos_img_1_path = os.path.join(cfg.OSVOS_RESULTS_FOLDERS_PATH, 
                                        sequence, frames[j+1][:5] + '.png')
        osvos_img_1 = cv2.imread(osvos_img_1_path)
        osvos_img_1_gray = cv2.imread(osvos_img_1_path, cv2.IMREAD_GRAYSCALE)
        
        # Create combined image
        _, combo_img_1, _, _ = compute_combo_img(contour_1_pred, osvos_img_1)
        combo_img_1_path = os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, 
                                        sequence, frames[j+1][:5] + '.png')
        if not os.path.exists(os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, sequence)):
            os.makedirs(os.path.join(cfg.COMBO_RESULTS_FOLDERS_PATH, sequence))

        cv2.imwrite(combo_img_1_path, combo_img_1*255)        
        
        # Load ground truth annotation
        annotation_1_path = os.path.join(raw_annotations_path, frames[j+1][:5] + '.png')
        annotation_1_gray = cv2.imread(annotation_1_path, cv2.IMREAD_GRAYSCALE)
        
        #Compute J
        J_combo = db_eval_iou(annotation_1_gray, combo_img_1)
        J_osvos = db_eval_iou(annotation_1_gray, osvos_img_1_gray)

        #Compute F
        F_combo = db_eval_boundary(combo_img_1, annotation_1_gray)
        F_osvos = db_eval_boundary(osvos_img_1_gray, annotation_1_gray)
        
        Js_combo.append(J_combo)
        Js_osvos.append(J_osvos)
        Fs_combo.append(F_combo)
        Fs_osvos.append(F_osvos)
        
        contour_0 = contour_1_pred.numpy()
    
    mean_J_combo = np.mean(np.array(Js_combo))
    mean_J_osvos = np.mean(np.array(Js_osvos))
    mean_F_combo = np.mean(np.array(Fs_combo))
    mean_F_osvos = np.mean(np.array(Fs_osvos))
    
    print('\tmean_J_combo: {}, mean_J_osvos: {}'.format(mean_J_combo, mean_J_osvos))
    print('\tmean_F_combo: {}, mean_F_osvos: {}'.format(mean_F_combo, mean_F_osvos))

    mean_Js_combo.append(mean_J_combo)
    mean_Js_osvos.append(mean_J_osvos)
    mean_Fs_combo.append(mean_F_combo)
    mean_Fs_osvos.append(mean_F_osvos)
    
mean_J_combo_overall = np.mean(np.array(mean_Js_combo))
mean_J_osvos_overall = np.mean(np.array(mean_Js_osvos))
mean_F_combo_overall = np.mean(np.array(mean_Fs_combo))
mean_F_osvos_overall = np.mean(np.array(mean_Fs_osvos))

print('mean_J_combo_overall: {}'.format(mean_J_combo_overall))
print('mean_J_osvos_overall: {}'.format(mean_J_osvos_overall))
print('mean_F_combo_overall: {}'.format(mean_F_combo_overall))
print('mean_F_osvos_overall: {}'.format(mean_F_osvos_overall))

#0: blackswan
	mean_J_combo: 0.8395457357938467, mean_J_osvos: 0.9437691704591111
	mean_F_combo: 0.8075487813712142, mean_F_osvos: 0.9702698124832759
#1: bmx-trees
	mean_J_combo: 0.22037807133923926, mean_J_osvos: 0.38081008359097557
	mean_F_combo: 0.34461347928037056, mean_F_osvos: 0.48947476296770015
#2: breakdance
	mean_J_combo: 0.3553168813383969, mean_J_osvos: 0.7125599532616964
	mean_F_combo: 0.42286625451253634, mean_F_osvos: 0.6924265529499325
#3: camel
	mean_J_combo: 0.7708185378295842, mean_J_osvos: 0.8466937428219212
	mean_F_combo: 0.7305475270234073, mean_F_osvos: 0.8605430893936815
#4: car-roundabout
	mean_J_combo: 0.9356199994178755, mean_J_osvos: 0.8780238057688655
	mean_F_combo: 0.8626135282202868, mean_F_osvos: 0.759978257371656
#5: car-shadow
	mean_J_combo: 0.8341168530776938, mean_J_osvos: 0.8857831255531767
	mean_F_combo: 0.788631648689179, mean_F_osvos: 0.874686049662771
#6: cows
	mean_J_combo: 0.8374029453863774, mean_J_osvos: 0.9524549806777443
	mean_F_combo: 0.8

IndexError: index 241 is out of bounds for dimension 2 with size 240